<a href="https://colab.research.google.com/github/AlexeyTri/MyMachineLearningSeminars/blob/main/seminar21/%D0%9E%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%A0%D0%B0%D0%BD%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обучение РАНЖИРОВАНИЮ

Библиотеки

In [5]:
!pip install --quiet -U pyserini jsonlines scikit-surprise fastrank

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = './data'
!mkdir data
!cp /content/drive/MyDrive/DATASET/MachineLearning/topicmodeling/lenta-ru-filtered.csv data
!cp /content/drive/MyDrive/DATASETS/MachineLearning/learningrank/MSLR-WEB10K.zip data/
!unzip data/MLSR-WEB10K.zip -d data/mlsr

In [8]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import gridspec

from tqdm.notebook import tqdm
import numpy as np
import  pandas as pd
import seaborn as sns
import torch
import scipy
import json
import re
import os
import jsonlines
import lightgbm as lgb

from surprise import Dataset, Reader, SVD
from fastrank import CDataset, TrainRequest
from sklearn.datasets import load_svmlight_file

from pyserini.search import SimpleSearcher

# Learning to Rank

Загружаем датасет

In [ ]:
DATA_PATH = 'data/mslr/Fold1'
(X_train, y_train, qid_train) = load_svmlight_file(
    os.path.join(DATA_PATH, 'train.txt'),
    dtype=np.float32,
    zero_based=False,
    query_id=True)
(X_val, y_val, qid_val) = load_svmlight_file(
    os.path.join(DATA_PATH, 'vali.txt'),
    dtype=np.float32,
    zero_based=False,
    query_id=True)
(X_test, y_test, qid_test) = load_svmlight_file(
    os.path.join(DATA_PATH, 'test.txt'),
    dtype=np.float32,
    zero_based=False,
    query_id=True)

train_size = 50000 
X_train = X_train.toarray()[:train_size]
y_train = y_train[:train_size]
qid_train = qid_train[:train_size]

val_size = 10000
X_val = X_val.toarray()[:val_size]
y_val = y_val[:val_size]
qid_val = qid_val[:val_size]


test_size = 10000
X_test = X_test.toarray()[:test_size]
y_test = y_test[:test_size]
qid_test = qid_test[:test_size]

In [ ]:
ids = np.random.RandomState(42).randint(0, len(X_train), size=5)
(X_train[ids][:, :6], y_train[ids], qid_train[ids])

Определяем модель

In [ ]:
train_request = TrainRequest.random_forest()
params = train_request.params
params.num_trees = 100
params.feature_sampling_rate = 0.5
params.insrance_sampling_rate = 0.5

params.quiet = True
params.seed = 42

Специальный формат данных

In [ ]:
dataset_train = CDataset.from_numpy(X_train, y_train, qid_train)
dataset_test = CDataset.from_numpy(X_test, y_test, qid_test)

features = dataset_train.feature_names()
train_queries = sorted(dataset_train.queries())
test_queries = sorted(dataset_test.queries())

train = dataset_train.subsample_queries(train_queries)
test = dataset_test.subsample_queries(test_queries)

In [ ]:
model = train.train_model(train_request)

\begin{align}
        \mathbf{DCG} @ \mathbf{K} &= \sum_{k=1}^K \frac{r_{true}( \pi^{-1}(k))}{\log_2(k+1)}
\end{align}

\begin{align}
        \mathbf{IDCG} @ \mathbf{K} &= \sum_{k=1}^K \frac{1}{\log_2(k+1)}
\end{align}

\begin{align}
        \mathbf{nDCG} @ \mathbf{K} &= \frac{\mathbf{DCG} @ \mathbf{K}}{{\mathbf{IDCG} @ \mathbf{K}}}
\end{align}



!TODO добавить описание формул

In [ ]:
test.evaluate(model, "NDCG@5")

# Рекомендательная система

In [ ]:
data = Dataset.load_builtin('ml-100k')
trainset =data.build_full_trainset()
dataframe = pd.DataFrame(
    trainset.all_ratings(),
    columns=['uid', 'iid', 'rating'])
dataframe.sample(5, random_state=42)

In [ ]:
algo = SVD
algo = algo.fit(trainset)

In [ ]:
(algo.predict('10', '4').est,
 algo.predict('10', '5').est,
 algo.predict('10', '6').est)

# Поисковая система

In [ ]:
data = pd.reaf_csv('data/lenta-ru-filtred.csv')

Генерация документов

In [ ]:
COLLECTION_DIR = 'data/collection/lenta'
if not os.path.exists(COLLECTION_DIR):
    os.makedirs(COLLECTION_DIR)

collection = data.values

batch_size = 32768
for ind in tqdm(range(0, len(collection), batch_size)):
    files = [{'id': ind+i,
              'contents': doc[0],
              'tag': doc[1],
              'date': doc[-1]}
             for i, doc in enumerate(collection[ind:ind+batch_size])]
    with jsonlines.open(os.path.join(COLLECTION_DIR,
                                     '{}.jsonl'.format(ind)),
                        mode='w') as writer:
        writer.write_all(files)

In [ ]:
with open(os.path.join(COLLECTION_DIR,
                       '{}.jsoinl'.format(ind))) as f:
    print('\n'.join(f.read().splitlines()[-5:]))

Построение индекса

In [ ]:
INDEX_DIR = 'data/index/lenta'
if not os.path.exists(INDEX_DIR):
    os.makedirs(INDEX_DIR)

In [ ]:
!python -m pyserini.index -collection JsonCollection -generator DefaultLuceneDocumentGenerator \
 -threads 16 -input {COLLECTION_DIR} \
 -index {INDEX_DIR} -storePositions -storeDocvectors -storeRaw

Используем поисковый индекс

ВМ-25

Пусть дан запрос Q, содержащий слова\begin{align}q_1,...,q_n\end{align} тогда функция ВМ-25 дает следующую оценку релевантности документа D запросу Q:




\begin{align} score(D,Q) = \sum_{i=1}^nIDF_{q_{i}} \cdot \frac{tf(q_{i},D) \cdot (k_{1}+1)}{tf(q_{i},D)+k_{1} \cdot(1-b+b\cdot\frac{|D|}{avgdl})} \end{align}

In [ ]:
searcher = SimpleSearcher(INDEX_DIR)
searcher.set_bm25()
hits = searcher.search('Поисковик', k=1)

for i in range(len(hits)):
    doc = searcher.doc(hits[i].docid)
    print(doc.raw())

QLD


Идея: латентное размещение Дирихле 

In [ ]:
searcher = SimpleSearcher(INDEX_DIR)
searcher.set_qld()
hits = searcher.search('Посиковик', k=1)

for i in range(len(hits)):
    doc = searcher.doc(hits[i].docid)
    print(doc.raw())

RM3

Пусть дан запрос Q, содержащий слова\begin{align}q_1,...,q_n\end{align} тогда функция ВМ-25 дает следующую оценку релевантности документа D запросу Q:



\begin{align} score(t,Q) = \alpha \cdot {softmax}\left(\sum_{d\in{PRD}}\left(\frac{tf(t,d)}{|d|}\cdot\prod_{i=1}^k\frac{tf(q_i,d)+\mu p(q_i)}{|q|+\mu}
\right)\right) + (1-\alpha)\cdot(\frac{tf(t,Q)}{|Q|})\end{align}



In [ ]:
searcher = SimpleSearcher(INDEX_DIR)
searcher.set_rm3()
hits = searcher.search('Посиковик', k=1)

for i in range(len(hits)):
    doc = searcher.doc(hits[i].docid)
    print(doc.raw())